### Comments:
 


In [0]:
using_colab = True

if using_colab :
  !git clone -l -s git://github.com/juanigp/CT-denoising.git cloned-repo
  %cd cloned-repo
  from google.colab import drive
  drive.mount('/gdrive')


import os
from IPython.core.debugger import set_trace
from models.Mini_REDCNN import REDCNN
from utils import utils
import torch
import torch.nn as nn
import torch.utils.data.sampler as sampler
from torch.autograd import Variable
from matplotlib import pyplot as plt
import random
import numpy as np

In [0]:
#wanna know how long does it take to train the net

from datetime import datetime
from dateutil.relativedelta import relativedelta

def time_diff(t_a, t_b):
    t_diff = relativedelta(t_b, t_a) 
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [0]:
#hyperparameters:
num_epochs = 1000
batch_size = 16
learning_rate = 0.0001

In [0]:
#instantiating the model:
model = REDCNN()
model.double()

#loss function
criterion = nn.MSELoss()

#optimizer algorithm
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

#if gpu available
if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
#dataset, dataloader

In [0]:
#dataset
if using_colab:
  csv_file = r'/gdrive/My Drive/patches/1.csv'  
else:
  csv_file = r'C:\Users\Juan Pisula\Desktop\ct_images\patches\100_FBPPhil_500FBP.csv'

dataset = utils.CTVolumesDataset(csv_file)

#this is no longer necessary
"""
#split of data in training data and validation data
num_samples = len(dataset)
training_samples_percentage = 0.8
split_index = int( num_samples * training_samples_percentage )
total_idx = list(range(num_samples))
random.seed(10)
random.shuffle(total_idx)
training_idx = total_idx[0 : split_index]
validation_idx = total_idx[split_index : num_samples]
training_sampler = sampler.SubsetRandomSampler(training_idx)
"""

#dataloader
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, suffle = True)


In [0]:
"""
#inspect training examples

batches = list(dataloader)

print(len(batches)) #how many batches
batch = batches[0]
print( len( batch ) ) #length of the batches (2 = lo res, hi res)
print( batch[0].size() ) #size of the lo res volumes of the batch: batch_size volumes, size of volume
plt.imshow(batch[0][1][10][:][:], cmap = 'gray' )

#enu = enumerate(dataloader)
#len(dataloader) # = amount of patches / batch size

(lo_res, hi_res) = batch
print(lo_res.size())
lo_res = lo_res.unsqueeze(1)
print(lo_res.size())
"""

In [0]:
#directory to save the model
if using_colab:
  models_dir = r'/gdrive/My Drive/models' 
else:
  models_dir = r'C:\Users\Juan Pisula\Desktop\ct_images'  

file_name = 'metrics.csv' 
file_dir = os.path.join(models_dir,file_name)

#set the model to train
model.train()
total_step = len(dataloader)

#train
start = datetime.now()

for epoch in range(num_epochs):
    #running_mse = 0.0
    #running_rmse = 0.0
    #running_psnr = 0.0
    for i, (lo_res, hi_res) in enumerate(dataloader):
        #add an extra dimension:
        lo_res = utils.var_or_cuda( lo_res.unsqueeze(1) )
        hi_res = utils.var_or_cuda(hi_res)
        if lo_res.size()[0] != batch_size:
            print("batch_size != {} drop last incompatible batch".format( batch_size ))
            continue
            
        #forward pass 
        outputs = model(lo_res)
        loss = criterion(outputs, hi_res.unsqueeze(1))
        #backward & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #record stats
        #note: record to .csv
        #running_mse += loss.item()
        #running_rmse += np.sqrt(loss.item())
        #psnr = 10 * np.log10(1 / loss.item()) #ver esto
        #running_psnr += psnr
        
        if (i+1) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    #save model after epoch   
    torch.save(model.state_dict(), os.path.join(models_dir, '3d_autoencoder_epoch_' + str(epoch) + '.pkl' ) )      
    
    #csv_line = str(running_mse) + ',' + str(running_psnr) + ',' + str(datetime.now()) + '\n'
    csv_line = str(loss.item()) + ',' + str(epoch) + '\n'
    with open(file_dir , 'a+') as file:
        file.write(csv_line)
        
        
stop = datetime.now()
dur = time_diff(start, stop)